In [1]:
print("Let's do object detection with YOLOv5!")

Let's do object detection with YOLOv5!


In [2]:
from ultralytics import YOLO

WARNING torchvision==0.20 is incompatible with torch==2.6.
Run 'pip install torchvision==0.21' to fix torchvision or 'pip install -U torch torchvision' to update both.
For a full compatibility table see https://github.com/pytorch/vision#installation


In [3]:
model = YOLO('yolov8m.pt')

100%|██████████| 49.7M/49.7M [00:31<00:00, 1.67MB/s]


In [4]:
import cv2

In [5]:
# Open the video file
video_path = 'sample_video.mp4'
cap = cv2.VideoCapture(video_path)

In [6]:
# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

In [7]:
# Define video writer for output
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_detected.mp4', fourcc, fps, (frame_width, frame_height))

In [8]:
# Process video frame by frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform object detection
    results = model.predict(frame)
    detections = results[0].boxes.data.cpu().numpy()

    for detection in detections:
        x1, y1, x2, y2, conf, cls = detection
        class_id = int(cls)
        label = model.names[class_id]
        # Draw bounding box and label
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        text = f"{label} ({conf:.2f})"
        cv2.putText(frame, text, (int(x1), int(y1) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    out.write(frame)


0: 384x640 (no detections), 61.9ms
Speed: 5.6ms preprocess, 61.9ms inference, 17.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 clock, 22.7ms
Speed: 2.2ms preprocess, 22.7ms inference, 124.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 bowls, 1 banana, 1 apple, 1 chair, 1 dining table, 1 clock, 22.6ms
Speed: 3.6ms preprocess, 22.6ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cups, 4 bowls, 1 banana, 1 apple, 1 chair, 1 dining table, 1 clock, 1 vase, 22.9ms
Speed: 1.8ms preprocess, 22.9ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cups, 2 bowls, 1 banana, 1 apple, 1 orange, 1 chair, 1 dining table, 1 clock, 1 vase, 22.6ms
Speed: 1.8ms preprocess, 22.6ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cups, 2 bowls, 1 banana, 1 apple, 1 orange, 1 chair, 1 dining table, 1 clock, 1 vase, 22.4ms
Speed: 1.8ms preprocess, 22.4ms inference, 2.7ms postpro

In [9]:
cap.release()
out.release()
cv2.destroyAllWindows()

In [12]:
import cv2
import time
from ultralytics import YOLO
from playsound import playsound
import threading

In [13]:
# Load the YOLOv8 model
model = YOLO("yolov8m.pt")

# Ensure 'apple' is in the model's class names
if 'apple' not in model.names.values():
    raise ValueError("The YOLO model does not include 'apple'. You may need a fruit-specific model.")

# Open input video
cap = cv2.VideoCapture('sample_video.mp4')

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Set up output video
out = cv2.VideoWriter('output_with_beep.mp4',
                      cv2.VideoWriter_fourcc(*'mp4v'),
                      fps, (frame_width, frame_height))


In [14]:
# Track apple presence status
apple_detected_prev = False

# Function to play beep sound (in separate thread)
def play_beep():
    playsound("beep-02.wav")  # Ensure this file exists in your current directory


In [17]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model.predict(frame)[0]
    detections = results.boxes.data.cpu().numpy()

    apple_detected_now = False

    for det in detections:
        x1, y1, x2, y2, conf, cls = det
        class_id = int(cls)
        label = model.names[class_id]
        
        if label.lower() == 'apple':
            apple_detected_now = True
            # Draw box and label
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)
            cv2.putText(frame, f"{label} {conf:.2f}", (int(x1), int(y1)-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    # Trigger beep if apple just appeared
    if apple_detected_now and not apple_detected_prev:
        threading.Thread(target=play_beep, daemon=True).start()

    apple_detected_prev = apple_detected_now
    out.write(frame)



0: 384x640 (no detections), 64.4ms
Speed: 4.8ms preprocess, 64.4ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 clock, 62.4ms
Speed: 3.1ms preprocess, 62.4ms inference, 4.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 bowls, 1 banana, 1 apple, 1 chair, 1 dining table, 1 clock, 62.9ms
Speed: 3.1ms preprocess, 62.9ms inference, 3.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cups, 4 bowls, 1 banana, 1 apple, 1 chair, 1 dining table, 1 clock, 1 vase, 63.0ms
Speed: 2.8ms preprocess, 63.0ms inference, 3.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cups, 2 bowls, 1 banana, 1 apple, 1 orange, 1 chair, 1 dining table, 1 clock, 1 vase, 62.6ms
Speed: 3.0ms preprocess, 62.6ms inference, 3.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cups, 2 bowls, 1 banana, 1 apple, 1 orange, 1 chair, 1 dining table, 1 clock, 1 vase, 62.8ms
Speed: 2.6ms preprocess, 62.8ms inference, 3.4ms postproces

In [5]:
!pip show moviepy


Name: moviepy
Version: 2.2.1
Summary: Video editing with Python
Home-page: 
Author: Zulko 2024
Author-email: 
License: MIT License
Location: C:\Users\asus\Desktop\Projects\recom_env\Lib\site-packages
Requires: decorator, imageio, imageio_ffmpeg, numpy, pillow, proglog, python-dotenv
Required-by: 


In [9]:
import cv2
import time
from ultralytics import YOLO
from playsound import playsound
import threading

WARNING torchvision==0.20 is incompatible with torch==2.6.
Run 'pip install torchvision==0.21' to fix torchvision or 'pip install -U torch torchvision' to update both.
For a full compatibility table see https://github.com/pytorch/vision#installation


In [15]:
from moviepy import VideoFileClip, AudioFileClip, CompositeAudioClip
import numpy as np

beep_timestamps = []
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model.predict(frame)[0]
    detections = results.boxes.data.cpu().numpy()

    apple_detected_now = False

    for det in detections:
        x1, y1, x2, y2, conf, cls = det
        class_id = int(cls)
        label = model.names[class_id]
        
        if label.lower() == 'apple':
            apple_detected_now = True
            # Draw box and label
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)
            cv2.putText(frame, f"{label} {conf:.2f}", (int(x1), int(y1)-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    # If apple just appeared, mark time for beep
    if apple_detected_now and not apple_detected_prev:
        timestamp_sec = frame_count / fps
        beep_timestamps.append(timestamp_sec)

    apple_detected_prev = apple_detected_now
    out.write(frame)
    frame_count += 1



0: 384x640 (no detections), 74.0ms
Speed: 4.4ms preprocess, 74.0ms inference, 25.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 clock, 62.8ms
Speed: 3.6ms preprocess, 62.8ms inference, 87.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 bowls, 1 banana, 1 apple, 1 chair, 1 dining table, 1 clock, 62.9ms
Speed: 3.8ms preprocess, 62.9ms inference, 4.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cups, 4 bowls, 1 banana, 1 apple, 1 chair, 1 dining table, 1 clock, 1 vase, 63.2ms
Speed: 3.0ms preprocess, 63.2ms inference, 3.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cups, 2 bowls, 1 banana, 1 apple, 1 orange, 1 chair, 1 dining table, 1 clock, 1 vase, 62.7ms
Speed: 2.9ms preprocess, 62.7ms inference, 3.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cups, 2 bowls, 1 banana, 1 apple, 1 orange, 1 chair, 1 dining table, 1 clock, 1 vase, 62.7ms
Speed: 2.8ms preprocess, 62.7ms inference, 3.6ms postproc

In [16]:
cap.release()
out.release()
cv2.destroyAllWindows()


In [42]:
from moviepy import VideoFileClip, AudioFileClip, CompositeAudioClip

# Load the video and the beep sound
video = VideoFileClip("output_with_beep.mp4")
beep = AudioFileClip("beep-02.wav")

# Define the timestamps where the beep should occur
beep_timestamps = [1.5, 3.0, 5.0]  # Example timestamps in seconds

# Create a list of beep audio clips starting at each timestamp
beeps = [beep.with_start(t) for t in beep_timestamps]

# Combine all beep sounds into a single audio track
composite_audio = CompositeAudioClip(beeps)

# Set the composite audio as the video's audio
final_video = video.with_audio(composite_audio)

# Write the final video to a file
final_video.write_videofile("final_with_beeps.mp4", codec="libx264", audio_codec="aac")

MoviePy - Building video final_with_beeps.mp4.
MoviePy - Writing audio in final_with_beepsTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
MoviePy - Writing video final_with_beeps.mp4



MoviePy - Done !
MoviePy - video ready final_with_beeps.mp4
